In [1]:
from PIL import Image
import torchvision.transforms.functional as TF
import visualpriors
import subprocess

import glob
import os

feature_type = 'normal'

root_dir = "../Pix3D/crop"
# Load image and rescale/resize to [-1,1] and 3x256x256
for category in os.listdir(root_dir):
    
    path = root_dir+'/'+category
    for _,_,list_name in os.walk(path):
        
        for name in list_name:
            im_path = path + '/' + name
            image = Image.open(im_path)
            o_t = TF.to_tensor(TF.resize(image, 224)) * 2 - 1
            o_t = o_t.unsqueeze_(0)

            # Transform to normals feature
            representation = visualpriors.representation_transform(o_t, feature_type, device='cpu') # phi(o_t) in the diagram below

            # Transform to normals feature and then visualize the readout
            pred = visualpriors.feature_readout(o_t, feature_type, device='cpu')
            
            
            if not os.path.exists('../Pix3D/normal/'+category):
                os.makedirs('../Pix3D/normal/'+category)
            # Save it
            TF.to_pil_image(pred[0] / 2. +0.5).save('../Pix3D/normal/{}/{}'.format(category,name))